In [5]:
!pip install pandas 

  Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl (11.3 MB)
   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.1 MB 5.7 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/13.1 MB 6.5 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/13.1 MB 9.7 MB/s eta 0:00:01
   ----------------------- ---------------- 7.6/13.1 MB 10.9 MB/s eta 0:00:01
   ----------------------------- ---------- 9.7/13.1 MB 10.8 MB/s eta 0:00:01
   -------------------------------------- - 12.6/13.1 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------- 13.1/13.1 MB 11.1 MB/s  0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
# Load dataset
df = pd.read_excel("../Data/Indian_EV_User_Spending_Behavior.xlsx")
print("Data Shape:", df.shape)
df.head()


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df['charging_duration_min'] = (df['Session_End'] - df['Session_Start']).dt.total_seconds() / 60
df['cost_efficiency'] = df['Total_Cost'] / df['Energy_Consumed_kWh']
df['peak_hour_flag'] = df['Session_Start'].dt.hour.between(18, 22).astype(int)
df['day_of_week'] = df['Session_Start'].dt.dayofweek


In [ ]:
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Summer"
    elif month in [6, 7, 8]:
        return "Monsoon"
    else:
        return "Autumn"

df['season'] = df['Month'].apply(get_season)


In [ ]:
monthly = df.groupby(['User_ID', 'Month', 'Year']).agg({
    'Energy_Consumed_kWh': ['sum', 'mean'],
    'Total_Cost': ['sum', 'mean'],
    'Wallet_Balance': 'mean',
    'charging_duration_min': 'mean',
    'cost_efficiency': 'mean',
    'peak_hour_flag': 'mean'
}).reset_index()

monthly.columns = [
    'User_ID', 'Month', 'Year',
    'total_kwh', 'avg_kwh',
    'total_cost', 'avg_cost',
    'avg_wallet_balance',
    'avg_session_duration',
    'avg_cost_efficiency',
    'peak_hour_ratio'
]

# Merge user info back
user_static = df[['User_ID', 'City', 'Vehicle_Type', 'Subscription_Type', 'Payment_Mode', 'Charger_Type']].drop_duplicates()
final_df = pd.merge(monthly, user_static, on='User_ID', how='left')
final_df.head()


,User_ID,Month,Year,total_kwh,avg_kwh,total_cost,avg_cost,avg_wallet_balance,avg_session_duration,avg_cost_efficiency,peak_hour_ratio,City,Vehicle_Type,Subscription_Type,Payment_Mode,Charger_Type
0,U00001,12,2024,6.12,6.12,72.46,72.46,336.08,60.0,11.839869,0.0,Hyderabad,2W,Basic,UPI,Superfast
1,U00002,6,2025,39.20,39.20,236.38,236.38,778.06,60.0,6.030102,0.0,Ahmedabad,4W,Basic,Credit Card,Slow
2,U00003,5,2025,27.11,27.11,184.35,184.35,1593.61,60.0,6.800074,0.0,Delhi,4W,Premium,UPI,Slow
3,U00004,2,2025,9.41,9.41,57.59,57.59,433.03,60.0,6.120085,0.0,Chennai,4W,Basic,Credit Card,Slow
4,U00005,1,2025,38.95,38.95,309.26,309.26,1788.99,60.0,7.939923,0.0,Delhi,2W,Pay-as-you-go,Credit Card,Superfast
